## Регуляризация

Ранее мы создали набор полиномиальных признаков и затем применили для них линейную регрессию. Но мы можем сделать модель ещё лучше, используя регуляризацию.

Регуляризация старается минимизировать метрику "сумма квадратов остатков" (RSS - residual sum of squares) *и* "функцию штрафа". Функция штрафа старается "наказывать" модели с очень большими коэффициентами. Некоторые методы регуляризации по сути приводят к тому, что некоторые неполезные признаки получают нулевые коэффициенты, то есть модель не будет использовать эти признаки.

Рассмотрим два метода регуляризации - гребневая регрессия (или ридж-регрессия, или регрессия риджа, англ. ridge regression) и регрессия лассо. Мы будем использовать их в сочетании с полиномиальными признаками.

### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Загрузка данных и подготовительные шаги

In [2]:
df = pd.read_csv("../DATA/Advertising.csv")
X = df.drop('sales',axis=1)
y = df['sales']

### Полиномиальная конвертация

In [3]:
from sklearn.preprocessing import PolynomialFeatures
polynomial_converter = PolynomialFeatures(degree=3,include_bias=False)
poly_features = polynomial_converter.fit_transform(X)

### Разбиение на обучающий и тестовый наборы данных -  Train | Test Split

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

### Масштабирование данных

Хотя в наших данных все признаки имеют один и тот же порядок величины (в $1000 долларов), это не всегда верно в других наборах данных. Поскольку математика регуляризации будет суммировать коэффициенты, очень важно стандартизировать признаки.

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Гребневая регрессия (Ridge Regression)

In [6]:
from sklearn.linear_model import Ridge
ridge_model = Ridge(alpha=10)
ridge_model.fit(X_train,y_train)
y_pred = ridge_model.predict(X_test)
from sklearn.metrics import mean_absolute_error,mean_squared_error
MAE = mean_absolute_error(y_test,y_pred)
MSE = mean_squared_error(y_test,y_pred)
RMSE = np.sqrt(MSE)

print('MAE =', MAE)
print('MSE =', MSE)
print('RMSE =', RMSE)



MAE = 0.5774404204714183
MSE = 0.8003783071528418
RMSE = 0.8946386461319686


Какова точность на обучающей выборке? (Мы будем использовать это позже для сравнения)

In [7]:
train_predictions = ridge_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.5288348183025331

### Выбираем значение alpha с помощью кросс-валидации

In [8]:
from sklearn.linear_model import RidgeCV
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0),scoring='neg_mean_absolute_error')
ridge_cv_model.fit(X_train,y_train)
ridge_cv_model.alpha_

0.1

Делаем прогноз

In [9]:
y_pred = ridge_cv_model.predict(X_test)
from sklearn.metrics import mean_absolute_error,mean_squared_error
MAE = mean_absolute_error(y_test,y_pred)
MSE = mean_squared_error(y_test,y_pred)
RMSE = np.sqrt(MSE)

print('MAE =', MAE)
print('MSE =', MSE)
print('RMSE =', RMSE)

MAE = 0.42737748843373746
MSE = 0.38201298815043333
RMSE = 0.6180719926921404


Метрика MAE на обучающем наборе

In [10]:
train_predictions = ridge_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.3094132105662787

## Регрессия Лассо

In [11]:
from sklearn.linear_model import LassoCV
lasso_cv_model = LassoCV(eps=0.1,n_alphas=100,cv=5)
lasso_cv_model.fit(X_train,y_train)
lasso_cv_model.alpha_

0.4943070909225828

In [12]:
y_pred = lasso_cv_model.predict(X_test)
from sklearn.metrics import mean_absolute_error,mean_squared_error
MAE = mean_absolute_error(y_test,y_pred)
MSE = mean_squared_error(y_test,y_pred)
RMSE = np.sqrt(MSE)

print('MAE =', MAE)
print('MSE =', MSE)
print('RMSE =', RMSE)

MAE = 0.6541723161252854
MSE = 1.2787088713079842
RMSE = 1.130800102276253


Метрика MAE на обучающем наборе

In [13]:
train_predictions = lasso_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.6912807140820695

## Elastic Net

Elastic Net сочетает в себе ридж-регрессию и лассо-регрессию, чтобы взять лучшее из обеих регуляризаций!

In [14]:
from sklearn.linear_model import ElasticNetCV
elastic_model = ElasticNetCV(l1_ratio=[.1, .5, .7,.9, .95, .99, 1],tol=0.01)
elastic_model.fit(X_train,y_train)
elastic_model.l1_ratio_

1.0

In [15]:
y_pred = elastic_model.predict(X_test)
from sklearn.metrics import mean_absolute_error,mean_squared_error
MAE = mean_absolute_error(y_test,y_pred)
MSE = mean_squared_error(y_test,y_pred)
RMSE = np.sqrt(MSE)

print('MAE =', MAE)
print('MSE =', MSE)
print('RMSE =', RMSE)

MAE = 0.5663262117569448
MSE = 0.5603340214638836
RMSE = 0.7485546215633724


Метрика MAE на обучающем наборе

In [16]:
train_predictions = elastic_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.4307582990472369

Из оценок видно, что эластичная сеть имеет бОльшую точность.